# Salary Predictions Based on Job Descriptions

### Problem Definition

As a Data Scientist in the recruitment industry, my goal is to build a system to predict salaries for a new set of job postings based on data provided on a set of historic job postings that include salaries.

Two CSV data files are available as a basis for training a machine learning model:

• train_features.csv: Each row represents metadata for an individual job posting.
The “jobId” column represents a unique identifier for the job posting. The remaining columns describe features of the job posting.
• train_salaries.csv: Each row associates a “jobId” with a “salary”.

The data upon which predictions should be made are stored in a further CSV data file:

• test_features.csv: Similar to train_features.csv, each row represents metadata for an individual job posting.

The output of my system should be a CSV file entitled test_salaries.csv where each row has the following format:

jobId, salary

### Library Imports

In [1]:
# Importing base libraries.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")

# Importing scikit-learn pre-processing libraries.
from sklearn.preprocessing import LabelEncoder


# Importing scikit-learn machine learning libraries.



# Author information.
__author__ = "Ross MacDonald"
__email__ = "ross.macdonald@technologist.com"

 ### Reusable Functions

In [2]:
# Load CSV data file with completion dialogue and shape confirmation.
def load_file(file):
    df =  pd.read_csv(file)
    shape = df.shape
    print("Data file is loaded, the shape of the dataset = {}".format(shape))
    return df

# Show first and last lines (number of lines defined by x) of a defined dataframe (df).
def ends(df, x):
    print('{} rows x {} columns'.format(np.shape(df)[0],np.shape(df)[1]))
    return df.head(x).append(df.tail(x))

# Produce plots for feature versus target.
def plot_feature(df, feature, target):
    fig = plt.figure(figsize=(16,16))

    
    # Produce ditribution plot of feature.
    plt.subplot(2,1,1)
    plt.tight_layout()
    if df[feature].dtype=='int64' or df[feature].dtype =='float64':
        sns.distplot(df[feature])
    else:
        df[feature].value_counts().sort_values().plot()
    plt.title('Distribution of '+feature, fontsize=13, weight='bold')
    plt.xlabel(feature, fontsize=13, weight='bold')
    plt.ylabel('Counts', fontsize=13, weight='bold')
    if feature =='companyId':
        plt.xticks(rotation=90)
    else:
        plt.xticks(rotation=45)
    
    # Produce plot that shows feature correlation with target.
    plt.subplot(2,1,2)
    plt.tight_layout()
    if df[feature].dtype=='int64' or df[feature].dtype=='float64':
        sns.regplot(x=feature,y=target,data=df)
    else:
        order = df.groupby(feature).mean().sort_values(target).reset_index()[feature].values
        sns.boxplot(x=feature,y=target,data=df,order=order)
    plt.title('Correlation of '+feature+' with '+ target, fontsize=13, weight='bold')
    plt.xlabel(feature, fontsize=13, weight='bold')
    plt.ylabel(target, fontsize=13, weight='bold')
    if feature =='companyId':
        plt.xticks(rotation=90)
    else:
        plt.xticks(rotation=45)

### Load the Data

In [4]:
# Load the data into pandas dataframes (csv files for training and test data).
train_feature_df = load_file('data/train_features.csv')
train_target_df = load_file('data/train_salaries.csv')
test_df = load_file('data/test_features.csv')

FileNotFoundError: [Errno 2] File b'data/train_features.csv' does not exist: b'data/train_features.csv'

In [ ]:
# Merge the data on jobId to get a single training dataset (includes features and target).
train_df = pd.merge(train_feature_df,train_target_df,how="inner",on="jobId")

In [ ]:
# Initial check of top 10 rows of train_df to confirm load and get an initial view on data / types.
train_df.head(10)

In [ ]:
# Initial check of top 10 rows of test_feature_df to confirm load and get an initial view on data / types.
test_df.head(10)

In [ ]:
# Get information on the train_feature_df dataframe. Ensure data types and number of entries per column are as expected.
train_df.info()

In [ ]:
# Get information on the train_feature_df dataframe. Ensure data types and number of entries per column are as expected.
test_df.info()

### Clean the Data

In [ ]:
# Check for duplicate rows in train_df.
print("Number of duplicated rows = {}".format(train_df.duplicated().sum()))

In [ ]:
# Check for duplicate rows in train_df.
print("Number of duplicated rows = {}".format(train_df.duplicated().sum()))

In [ ]:
# Check for null values in the columns of train_df.
train_df.isnull().sum().to_frame('Null Entries')

In [ ]:
# Check for null values in the columns of test_df.
test_df.isnull().sum().to_frame('Null Entries')

##### There are no duplicates or null entries in the data, so no need to drop rows / entries or subsitute any values. 

In [ ]:
# Get the column names for numerical columns from train_df to enable us to filter for invalid values. 
print(train_df.select_dtypes(include=['float64', 'int64']).columns.values)

In [ ]:
# Having found the column names for numerical columns from train_df, find rows that contain invalid values (i.e. <0 or <=0).
print(train_df[(train_df['yearsExperience'] < 0) | (train_df['milesFromMetropolis'] < 0) | (train_df['salary'] <= 0)])

In [ ]:
# Remove the rows that contain invalid data from the train_df dataframe (i.e. salary was invalid in all cases, thus only include rows with salaries more than 0).
train_df = train_df[train_df.salary > 0]

In [ ]:
# Check dataframe information to confirm that the expected rows have been dropped.
train_df.info()

In [ ]:
# Reset the index of train_df after dropping the invalid values and print head and tail (using function ends) to confirm.
# reindex. Makes rows sequential to prevent any misunderstanding in subsequent analysis.
train_df = train_df.reset_index(drop=True)
ends(train_df, 3)

##### Invalid numerical entries have been dropped from the train_df dataframe, ensuring only valid data informs the prediction model / system. Index has been reset to prevent any misunderstanding (due to non-sequential rows in data) in any further analysis.

In [ ]:
# Visualize target variable (salary).
plt.figure(figsize = (14, 6))
plt.subplot(1,2,1)
sns.boxplot(train_df.salary)
plt.subplot(1,2,2)
sns.distplot(train_df.salary, bins=50 , color = 'green')
plt.show()

##### From initially observing the data graphically, it appears that there are outliers on the upper end (invalid zero entries have been removed, these would have been lower end outliers). My next action based on this insight is to use summary statistics and calculated IQR to delve into the outliers further.

In [ ]:
# Generate summary statistics and IQR for the training data train_df. Use IQR to calculate the upper and lower limits.
train_df.salary.describe()
summary_stats = train_df.salary.describe()
print(summary_stats)
IQR = summary_stats['75%'] - summary_stats['25%']
upper = summary_stats['75%'] + 1.5 * IQR
lower = summary_stats['25%'] - 1.5 * IQR
print('The upper and lower limits calculated to confirm suspected outliers are {} and {}.'.format(upper, lower))

##### The summary statistics allow us to access the 25th and 75th percentiles of the salary column of train_df. IQR is calculated and presented above by subtracting the 75th percentile from the 25th percentile value. The IQR value is then used to calculate the lower limit and the upper limit of the salary column data, any data lying outside of these limits can be considered an outlier.

In [ ]:
# Confirm for sure that there are no outliers (rows) equal to or below the lower limit.
train_df[train_df.salary < 8.5]

In [ ]:
# Filter the train_df dataframe to show the rows that contain a salary more than the upper limit. 
train_df[train_df.salary > 220.5]

In [ ]:
# Calculate the upper limit outliers count as a percentage of the salary count (calculate salary and outlier count first).
training_salary_count = train_df.salary.count()
upper_outliers_count = train_df[train_df.salary > 220.5].salary.count()
upper_outlier_percentage = upper_outliers_count / training_salary_count * 100
upper_outlier_percentage = upper_outlier_percentage.round(2)
print('The upper outliers count ({}) is {}% of the salary count ({}).'.format(upper_outliers_count, upper_outlier_percentage, training_salary_count))

##### The upper outliers constitute 0.71% of the data. This is a relatively low volume of data (7117 rows), given the large data set we have (999995 rows). My next course of action is to remove the outlier rows in an effort to increase the quality of the predictions going forward. For guidance on my thought process and justification for removing the outliers see https://humansofdata.atlan.com/2018/03/when-delete-outliers-dataset/.

In [ ]:
# Remove the upper outliers from the training data, reindex the training data and output head and tail to confirm rows have been dropped.
# Drop jobId column as it is a unique identifier and will not contribute to prediction going forward
train_df = train_df[train_df.salary <= 220.5]
train_df.drop('jobId', axis=1, inplace=True)
train_df = train_df.reset_index(drop=True)
ends(train_df, 3)

##### Data cleansing steps are complete. Invalid data has been removed and outlier analysis and removal has been carried out.

### Explore the data (EDA)

In [ ]:
plot_feature(train_df,'companyId','salary')

##### The plots for companyId illustrate that there is no correlation between companyId and salary. The IQR's are similar across companies. This suggests companyId would not be a candidate for predicting salary.

In [ ]:
plot_feature(train_df,'jobType','salary')

##### The plots for jobType illustrate that there is a correlation between jobType and salary. The IQR's differ across jobTypes. As perhaps expected, CEO are paid highest (at the top of the IQR) and janitors are paid least (at the bottom of the IQR). Salary appears to adhere to jobType seniority, this suggests jobType may be a good candidate for predicting salary.

In [ ]:
plot_feature(train_df,'degree','salary')

##### The plots for degree illustrate that there is a correlation between degree and salary. The IQR's differ across degree. Doctoral level employees are paid highest (at the top of the IQR) and employees without a degree are paid least (at the bottom of the IQR - only a little lower than high school educated employees). Salary appears to adhere to education level, this suggests degree may be a good candidate for predicting salary.

In [ ]:
plot_feature(train_df,'major','salary')

##### The plots for major illustrate that there is a correlation between major and salary. The IQR's differ across major. Engineering major employees are paid highest (at the top of the IQR) and employees without a major are paid least (at the bottom of the IQR - unclear whether this is empoyees with degrees without majors, without degrees or a combination of both). Salary appears to adhere to degree major, this suggests major may be a good candidate for predicting salary.

In [ ]:
plot_feature(train_df,'industry','salary')

##### The plots for industry illustrate that there is a correlation between industry and salary. The IQR's differ across industry. Oil, and finance employees are paid highest (at the top of the IQR) and employees in education are paid least (at the bottom of the IQR. This notable differences between industry suggest it may be a good candidate for predicting salary.

In [ ]:
plot_feature(train_df,'yearsExperience','salary')

##### The plots for yearsExperience illustrate that there is a correlation between yearsExperience and salary. The trend is that the mean salary increases as yearsExperience increase. This suggests that yearsExperience may be a good candidate for predicting salary.

In [ ]:
plot_feature(train_df,'milesFromMetropolis','salary')

##### The plots for milesFromMetropolis illustrate that there is a correlation between milesFromMetropolis and salary. The trend is that the mean salary decreases as milesFromMetropolis increase. This suggests that milesFromMetropolis may be a good candidate for predicting salary.

In [ ]:
# Based on the above analysis, companyId will be dropped, as it will not be taken forward as a feature for prediction.
train_df.drop('companyId', axis=1, inplace=True)
train_df.info()

In [ ]:
# Encode for heatmap
# https://github.com/ankur26/SalaryPrediction/blob/master/Salary%20Prediction%20Notebook.ipynb

In [ ]:
train_df_cat = train_df
train_df_cat['jobType'] =train_df_cat['jobType'].astype('category').cat.codes
train_df_cat['degree'] =train_df_cat['degree'].astype('category').cat.codes
train_df_cat['major'] =train_df_cat['major'].astype('category').cat.codes
train_df_cat['industry'] =train_df_cat['industry'].astype('category').cat.codes

In [ ]:
# One-hot encode categorical data and assign to new train_df_cat dataset.
# train_df_cat = pd.get_dummies(train_df, sparse=True)

In [ ]:
# Check the resulting datafarme.
# train_df_cat.info()

In [ ]:
# Using train_df_cat, produce a heatmap to observe the correlation between features.
# plt.figure(figsize=(80,80))
# c= train_df_cat.corr()
# sns.heatmap(c,cmap='coolwarm',annot=True, linewidth=0.5)
# c

In [ ]:
#summarize each feature variable
# Finding the relations between the variables.
#summarize the target variable
#look for correlation between each feature and the target
#look for correlation between features

### ---- 5 Establish a baseline ----

In [ ]:
#select a reasonable metric (MSE in this case)
#create an extremely simple model and measure its efficacy
#e.g. use "average salary" for each industry as your model and then measure MSE
#during 5-fold cross-validation

### ---- 6 Hypothesize solution ----

In [ ]:
#brainstorm 3 models that you think may improve results over the baseline model based
#on your 

Brainstorm 3 models that you think may improve results over the baseline model based on your EDA and explain why they're reasonable solutions here.

Also write down any new features that you think you should try adding to the model based on your EDA, e.g. interaction variables, summary statistics for each group, etc

## Part 3 - DEVELOP

You will cycle through creating features, tuning models, and training/validing models (steps 7-9) until you've reached your efficacy goal

#### Your metric will be MSE and your goal is:
 - <360 for entry-level data science roles
 - <320 for senior data science roles

### ---- 7 Engineer features  ----

In [ ]:
#make sure that data is ready for modeling
#create any new features needed to potentially enhance model

### ---- 8 Create models ----

In [ ]:
#create and tune the models that you brainstormed during part 2

### ---- 9 Test models ----

In [ ]:
#do 5-fold cross validation on models and measure MSE

### ---- 10 Select best model  ----

In [ ]:
#select the model with the lowest error as your "prodcuction" model

## Part 4 - DEPLOY

### ---- 11 Automate pipeline ----

In [ ]:
#write script that trains model on entire training set, saves model to disk,
#and scores the "test" dataset

### ---- 12 Deploy solution ----

In [ ]:
#save your prediction to a csv file or optionally save them as a table in a SQL database
#additionally, you want to save a visualization and summary of your prediction and feature importances
#these visualizations and summaries will be extremely useful to business stakeholders

### ---- 13 Measure efficacy ----

We'll skip this step since we don't have the outcomes for the test data